In [1]:
import numpy as np
import matplotlib as plt
import pandas as pd

In [2]:
#read csv files that contain the training and testing data
DS2_train=pd.read_csv("out/A2.4/DS2_train.csv",header=None)
DS2_train=np.array(DS2_train)
DS2_test=pd.read_csv("out/A2.4/DS2_test.csv",header=None)
DS2_test=np.array(DS2_test)

In [3]:
#learn parameters using maximum likelihood approach
N0=0
N1=0
m0=np.zeros([1,20])
m1=np.zeros([1,20])

for j in range(0,2800): 
    if(DS2_train[j,20]==1):
        N1=N1+1
        m1=m1+DS2_train[int(j),:20]
    else:
        N0=N0+1
        m0=m0+DS2_train[int(j),:20]
     
    
pi=float(N0)/(N0+N1)
m0=np.divide(m0,N0)
m1=np.divide(m1,N1)

s0=np.zeros([20,20])
s1=np.zeros([20,20])



for k in range(0,2800):
    if(DS2_train[k,20]==1):
        s1=np.add(s1,np.matmul((DS2_train[k,0:20]-m1).T,((DS2_train[k,0:20]-m1))))
        
    else:
        s0=np.add(s0,np.matmul((DS2_train[k,0:20]-m0).T,((DS2_train[k,0:20]-m0))))

s0=s0/float(N0)
s1=s1/float(N1)
s=pi*s0+(1-pi)*s1

In [4]:
#build up the model's parameter and define the sigmoid function
def sigmoid(a):
    return 1.0/(1.0+np.exp(-a))

w0_0=-0.5*m0.dot(np.linalg.inv(s)).dot(m0.T)+0.5*m1.dot(np.linalg.inv(s)).dot(m1.T)+np.log(pi/(1-pi))
w_0=np.linalg.inv(s).dot((m0-m1).T)

w0_1=-0.5*m1.dot(np.linalg.inv(s)).dot(m1.T)+0.5*m0.dot(np.linalg.inv(s)).dot(m0.T)+np.log((1-pi)/pi)
w_1=np.linalg.inv(s).dot((m1-m0).T)

print "w0_0: ", np.ravel(w0_0),"\n","w0_1: ", np.ravel(w0_1),"\n","w_0: ","\n",np.ravel(w_0),"\n","w_1:","\n",np.ravel(w_1)

w0_0:  [ 0.07782143] 
w0_1:  [-0.07782143] 
w_0:  
[ 0.03095733  0.00568334  0.04279177  0.05273383 -0.04133535  0.09384691
  0.0326761  -0.02106459 -0.03836051 -0.09044405 -0.01191598 -0.01282974
 -0.03690949 -0.01060537  0.01106433 -0.01245718  0.06254451 -0.01113789
 -0.01637733 -0.07398869] 
w_1: 
[-0.03095733 -0.00568334 -0.04279177 -0.05273383  0.04133535 -0.09384691
 -0.0326761   0.02106459  0.03836051  0.09044405  0.01191598  0.01282974
  0.03690949  0.01060537 -0.01106433  0.01245718 -0.06254451  0.01113789
  0.01637733  0.07398869]


In [5]:
#Compute the best fit accuracy, precision, recall and F-measure
tp=0
tn=0
fp=0
fn=0
for r in range(0,1200):
    p0=sigmoid(w0_0+w_0.T.dot(DS2_test[r,0:20]))
    p1=sigmoid(w0_1+w_1.T.dot(DS2_test[r,0:20]))
    if (p1>p0):
        if(DS2_test[r,20]):
            tp=tp+1
        else:
            fp=fp+1
    else:
        if(DS2_test[r,20]):
            fn=fn+1
        else:
            tn=tn+1

acc=(tp+tn)/1200.0            
pre=tp/float(tp+fp)
rec=tp/float(tp+fn)
f_mes=(2.0*pre*rec)/float(pre+rec)

print tp+fp+tn+fn
print "Accuracy: ", acc, "\n","Precision: ", pre, "\n","Recall: ", rec, "\n","F_measure: ", f_mes

1200
Accuracy:  0.5275 
Precision:  0.54351687389 
Recall:  0.496753246753 
F_measure:  0.519083969466
